In [1]:
import time

from openai import OpenAI
from homeworks.utils.aidevsApi import get_auth_token, get_task, send_answer, API_KEY_OPENAI

client = OpenAI(api_key=API_KEY_OPENAI)

token = get_auth_token('whoami')
task = get_task(token, print_task=True)

----------- Task description -----------
msg: Each time you call up this task, I will return a trivia item about a certain person (the person does not change). Guess who I am
hint: Był pod wpływem Zen buddyzmu i odwiedził Indie w poszukiwaniu duchowego oświecenia
-----------    ----------    -----------


In [2]:
def guess_person(hints):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant who need to guess who is MYSTERY PERSON."
                                      "If you are not sure for 100% write me NO."
                                      "Only if you know answer for 100% write name and surname of this mystery person."
                                      "User each time will give you new hint."},
        {"role": "user", "content": f"I have information about a person: MYSTERY PERSON. Do you know who this person is?"}
    ]

    for hint in hints:
        # add all hints to conversation
        conversation.append({"role": "assistant", "content": f"hint: {hint}"})

    conversation.append({"role": "user", "content": f"Do you know who MYSTERY PERSON is?"})

    # ask OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        max_tokens=100
    )

    #answer = response['choices'][0]['message']['content']
    answer = response.choices[0].message.content

    # check answer
    if "NO" in answer.upper():
        # if not then continue
        print(f"Assistant is not sure. Trying another hint :)")
        time.sleep(2)
        return 0
    else:
        print(answer)
        send_answer(token, answer)
        return 1

In [3]:
hints = []

for i in range(9):
    # get information about task from DevAI2
    token = get_auth_token('whoami')
    hint = get_task(token, print_task=False)['hint']
    hints.append(hint)
    print(f"Try {i+1}. Hint: {hint}")

    # try to guess
    guess = guess_person(hints)
    if guess:
        break

Try 1. Hint: był wegetarianinem i eksperymentował z różnymi ekstremalnymi dietami
Assistant is not sure. Trying another hint :)
Try 2. Hint: W momencie śmierci, jego majątek był szacowany na około 10,2 miliarda dolarów
Yes, the MYSTERY PERSON is Steve Jobs.
Sending answer: done! :)
